<a href="https://colab.research.google.com/github/JayKarhade/MI_VCG_DL/blob/main/MI_Localization_mode_newfrequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import glob 
import PIL
import sklearn
import numpy as np
import pandas as pd
import scipy.io as sio 
import matplotlib.pyplot as plt
import warnings 
import cv2
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential,Model
from keras.layers import Input, Dense, Activation, Flatten, Dropout,BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
from sklearn.utils import shuffle
from PIL import Image
from tensorflow.keras.regularizers import l2
#from keras.utils import to_categorical
from scipy.io import loadmat
from keras import layers
import keras
from keras import backend as K
from sklearn.model_selection import StratifiedKFold

**dataset preparation step**

In [2]:
'''
path1 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/anterior"
path2 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/anterolateral"
path3 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/anteroseptal"
path4 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/inferior"
path5 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/inferolateral"
path6 = "/content/drive/MyDrive/mvmd_vcg_mi_detection/infero_postero_lateral"
my_list_class1 = os.listdir(path1)
my_list_class2 = os.listdir(path2)
my_list_class3 = os.listdir(path3)
my_list_class4 = os.listdir(path4)
my_list_class5 = os.listdir(path5)
my_list_class6 = os.listdir(path6)

data_load = []
sum = 0
y = []
for i in range(len(my_list_class1)):
  address = path1+'/'+my_list_class1[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(0)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class2)):
  address = path2+'/'+my_list_class2[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(1)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class3)):
  address = path3+'/'+my_list_class3[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(2)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class4)):
  address = path4+'/'+my_list_class4[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(3)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class5)):
  address = path5+'/'+my_list_class5[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(4)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class6)):
  address = path6+'/'+my_list_class6[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(5)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

data_load = np.asarray(data_load)
y = np.asarray(y)
print(y.shape)
print(data_load.shape)

dataset = np.zeros([y.shape[0],15,651])

for i in range(dataset.shape[0]):
  for j in range(3):
    for k in range(5):
      dataset[i,(5*j) + k] = data_load[i][j][k]

data = np.zeros([y.shape[0],651,15])
for i in range(dataset.shape[0]):
  data[i] = dataset[i].T

np.save("/content/drive/MyDrive/mvmd_vcg_mi_detection/localization_label.npy",y)
np.save("/content/drive/MyDrive/mvmd_vcg_mi_detection/localization_data.npy",data)
'''

(14421,)
(14421, 3)


**Run from here**

In [2]:
data = np.load("/content/drive/MyDrive/mvmd_vcg_mi_detection/localization_data.npy")
y = np.load("/content/drive/MyDrive/mvmd_vcg_mi_detection/localization_label.npy")

In [3]:
data.shape

(14421, 651, 15)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)


y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],6))

for i in range(np.array(y_train).shape[0]):
  label = y_train[i]
  y_tr_one_hot[i][int(label)] = 1

y_val_one_hot  = np.zeros((np.array(y_val).shape[0],6))

for i in range(np.array(y_val).shape[0]):
  label = y_val[i]
  y_val_one_hot[i][int(label)] = 1


In [7]:
##Define model
def model_define():
  model=Sequential()
  model.add(Conv1D(filters=20, kernel_size=5, activation='relu',input_shape=(651,15)))
  model.add(MaxPooling1D(pool_size=3, strides = 3))
  model.add(Conv1D(filters=60, kernel_size=5, activation='relu'))
  #model.add(Dropout(0.7))
  model.add(MaxPooling1D(pool_size=3, strides = 5))
  model.add(Conv1D(filters=120, kernel_size=5,))
  model.add(Conv1D(filters=120,kernel_size=7,))
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(Dense(700, activation='relu'))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(6,activation='sigmoid'))
  return model

k_sum = 0
f1_sum = 0
acc_sum = 0
sens_sum = 0
spec_sum = 0

for i in range(5):
  print("Trial:",i)
  # x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=1)
  # x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)


  # y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],6))

  # for i in range(np.array(y_train).shape[0]):
  #   label = y_train[i]
  #   y_tr_one_hot[i][int(label)] = 1

  # y_val_one_hot  = np.zeros((np.array(y_val).shape[0],6))

  # for i in range(np.array(y_val).shape[0]):
  #   label = y_val[i]
  #   y_val_one_hot[i][int(label)] = 1
  K.clear_session()
  modelf = model_define()

  #print(modelf.summary())
  optimizer = keras.optimizers.Adam(lr=0.00004)
  modelf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  modelf.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot), epochs=15, batch_size=1024)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],6))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = modelf.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  acc_sum = acc_sum + test_acc

  ##Evaluating Sensitivity, Accuracy and Kappa scores

  Y_pred = modelf.predict_classes(x_test)

  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  k_sum = k_sum+K_cappa
  print("cohen kappa scores:" ,K_cappa)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  f1_sum = f1_sum + F1
  print("F1 scores:", F1)

  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)

  print(classification_report(y_test, Y_pred))

Trial: 0


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
12/12 [==============================] - 48s 297ms/step - loss: 1.4052 - accuracy: 0.4106 - val_loss: 0.4309 - val_accuracy: 0.8226
Epoch 2/15
12/12 [==============================] - 1s 113ms/step - loss: 0.3447 - accuracy: 0.8814 - val_loss: 0.1687 - val_accuracy: 0.9304
Epoch 3/15
12/12 [==============================] - 1s 113ms/step - loss: 0.1120 - accuracy: 0.9588 - val_loss: 0.0661 - val_accuracy: 0.9754
Epoch 4/15
12/12 [==============================] - 1s 113ms/step - loss: 0.0378 - accuracy: 0.9893 - val_loss: 0.0150 - val_accuracy: 0.9957
Epoch 5/15
12/12 [==============================] - 1s 113ms/step - loss: 0.0113 - accuracy: 0.9974 - val_loss: 0.0071 - val_accuracy: 0.9975
Epoch 6/15
12/12 [==============================] - 1s 113ms/step - loss: 0.0066 - accuracy: 0.9986 - val_loss: 0.0063 - val_accuracy: 0.9963
Epoch 7/15
12/12 [==============================] - 1s 112ms/step - loss: 0.0022 - accuracy: 0.9993 - val_loss: 0.0103 - val_accuracy: 0.9969
Epoch

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


cohen kappa scores: 0.998317462131238
F1 scores: 0.9986139986139986
confusion matrix 
 [[170   0   0   0   0   0]
 [  0 180   0   0   0   0]
 [  0   2 324   0   0   0]
 [  0   0   0 275   0   0]
 [  0   0   0   0 285   0]
 [  0   0   0   0   0 207]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       170
           1       0.99      1.00      0.99       180
           2       1.00      0.99      1.00       326
           3       1.00      1.00      1.00       275
           4       1.00      1.00      1.00       285
           5       1.00      1.00      1.00       207

    accuracy                           1.00      1443
   macro avg       1.00      1.00      1.00      1443
weighted avg       1.00      1.00      1.00      1443

Trial: 1
Epoch 1/15
12/12 [==============================] - 3s 166ms/step - loss: 1.5152 - accuracy: 0.3620 - val_loss: 0.6131 - val_accuracy: 0.7899
Epoch 2/15
12/12 [==============================] - 1s 1

In [8]:
print("Average holdout accuracy score :",acc_sum/5)
print("Average holdout kappa score :",k_sum/5)
print("Average holdout f1 score :",f1_sum/5)
print("Average holdout sensitivity score :",sens_sum/5)
print("Average holdout specificity score :",spec_sum/5)

Average holdout accuracy score : 0.9986139893531799
Average holdout kappa score : 0.9927624946081547
Average holdout f1 score : 0.9940401940401941
Average holdout sensitivity score : 0.0
Average holdout specificity score : 0.0


**K fold Cross Validation**

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data_cv = data[indices]
y_cv = y[indices]

In [ ]:
y_one_hot  = np.zeros((np.array(y_cv).shape[0],6))

for i in range(np.array(y_cv).shape[0]):
  label = y_cv[i]
  y_one_hot[i][int(label)] = 1

In [ ]:
##Cross Validation
folds=10
k_cv_sum = 0
f1_cv_sum = 0
acc_cv_sum = 0
sens_cv_sum = 0
spec_cv_sum = 0

kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)
#cvscores = []

for train,test in kfold.split(data_cv,y_cv):
  model=Sequential()
  model.add(Conv1D(filters=20, kernel_size=5, activation='relu',input_shape=(651,15)))
  model.add(MaxPooling1D(pool_size=3, strides = 3))
  model.add(Conv1D(filters=60, kernel_size=5, activation='relu'))
  #model.add(Dropout(0.7))
  model.add(MaxPooling1D(pool_size=3, strides = 5))
  model.add(Conv1D(filters=120, kernel_size=5,))
  model.add(Conv1D(filters=120,kernel_size=7,))
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(Dense(700, activation='relu'))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(6,activation='sigmoid'))
  
  opt = keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  #Train
  model.fit(x_cv[train], y_one_hot[train],batch_size=256,epochs=15,verbose=1)
  #Test
  test_loss, test_acc = model.evaluate(data_cv[test],y_one_hot[test],batch_size=8, verbose=1)
  #Metrics
  print('Accuracy:',test_acc)
  acc_cv_sum = acc_cv_sum + test_acc
  Y_pred = model.predict_classes(data_cv[test])

  K_cappa = sklearn.metrics.cohen_kappa_score(y_cv[test],Y_pred)
  k_cv_sum = k_cv_sum+K_cappa
  print("cohen kappa scores:" ,K_cappa)

  F1 = sklearn.metrics.f1_score(y_cv[test],Y_pred,average="micro")
  f1_cv_sum = f1_cv_sum + F1
  print("F1 scores:", F1)

  cm1 = confusion_matrix(y_cv[test],Y_pred)
  print("confusion matrix \n",cm1)

  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  sens_cv_sum = sens_cv_sum + sensitivity1
  print('Sensitivity : ', sensitivity1 )

  specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  spec_cv_sum = spec_cv_sum + specificity1
  print('Specificity : ', specificity1)

  #cvscores.append(test_acc)
  K.clear_session()

In [ ]:
print("Average 10 cv  accuracy score :",acc_cv_sum/10)
print("Average 10 cv  kappa score :",k_cv_sum/10)
print("Average 10 cv  f1 score :",f1_cv_sum/10)
print("Average 10 cv  sensitivity score :",sens_cv_sum/10)
print("Average 10 cv  specificity score :",spec_cv_sum/10)